# Parte 2 - Completar dataset de accidentes de aviones mediante scrapping

In [1]:
import pandas as pd

In [657]:
data = pd.read_csv("Airplane_Crashes_and_Fatalities_Since_1908.csv.zip", compression = 'zip')
print 'Numero de filas:',len(data)
data.head()

Numero de filas: 5268


,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary
0,09/17/1908,17:18,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,1,2.0,1.0,0.0,"During a demonstration flight, a U.S. Army fly..."
1,07/12/1912,06:30,"AtlantiCity, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,NaN,5.0,5.0,0.0,First U.S. dirigible Akron exploded just offsh...
2,08/06/1913,NaN,"Victoria, British Columbia, Canada",Private,-,NaN,Curtiss seaplane,NaN,NaN,1.0,1.0,0.0,The first fatal airplane accident in Canada oc...
3,09/09/1913,18:30,Over the North Sea,Military - German Navy,NaN,NaN,Zeppelin L-1 (airship),NaN,NaN,20.0,14.0,0.0,The airship flew into a thunderstorm and encou...
4,10/17/1913,10:30,"Near Johannisthal, Germany",Military - German Navy,NaN,NaN,Zeppelin L-2 (airship),NaN,NaN,30.0,30.0,0.0,Hydrogen gas which was being vented was sucked...


Vamos a intentar añadir una nueva columna al dataframe, denominada 'Role', que indique el cometido para el que el tipo de avión está destinado ('bombardero', 'trasporte militar', 'transporte de pasajeros', ...)

Vamos a tomar como ejemplo el enlace del enunciado de la prueba:
'https://en.wikipedia.org/wiki/Breguet_14'
En la tabla de la derecha, aparece una tabla con información acerca del avión, como su cometido, su fabricante, las aerolíneas que lo han usado, etc. En concreto, para el Breguet 14, su cometido fue el de bombardero. La idea es extraer esta información mediante scrapping.

En Chrome, podemos acceder al código de la página, pichando con el botón derecho del ratón sobre la página y seleccionando inspeccionar. Usando la biblioteca BeautifulSoup, podemos acceder al contenido de la tabla y extraer la información que necesitamos de manera rápida.

In [622]:
from bs4 import BeautifulSoup
from urllib2 import urlopen

url = "https://en.wikipedia.org/wiki/Breguet_14"
soup = BeautifulSoup(urlopen(url).read(), "lxml")
table = soup.find('table')
table_headers = table.find_all('th')
for h in table_headers:
    if 'Role' in h:
        row = h.find_parent()
str(row.getText()).split('\n')[2]

'Bomber'

Ahora tendríamos que repetir este procedimient para cada tipo de avión. Vamos a calcular el número de tipos diferentes de avión que aparecen en el dataset.

In [625]:
len(data['Type'].unique())

2447

Nos encontramos con la problemática de que para cinco mil registros de accidentes aéreos, parece que han sido cometidos por dos mil quinientos tipos de aviones diferentes!! Lo más probable es que los valores de la columna 'Type' no sean consistentes y el mismo tipo de avión aparezca registrado con diferente nomenclatura.

Por ejemplo, vamos a ver las inconsistencias en nomenclatura para los aviones 'Antonov'

In [638]:
antonov = data[data['Type'].str.contains('Antonov') == True]['Type'].unique()
print len(antonov)
print antonov

76
['Antonov AN-9' 'Antonov AN-10' 'Antonov AN-10A' 'Antonov An-10A'
 'Antonov AN-12' 'Antonov AN-24B' 'Antonov AN-24' 'Antonov AN-12BP'
 'Antonov An-12 - Ilyshin IL-14' 'Antonov AN-24V' 'Antonov An-12PL'
 'Antonov An-12TB' 'Antonov AN-22' 'Antonov An-12B' 'Antonov An-10'
 'Antonov AN-26' 'Antonov An-24B' 'Antonov AN-12V' 'Antonov An-24'
 'Antonov AN-24RV' 'Antonov AN-24 / Yakovlev Yak-40' 'Antonov An-12BP'
 'Antonov An-12' 'Antonov AN-24N' 'Antonov 2PF'
 'Antonov AN-24RV / Soviet Air Force TU-16' 'Antonov An-24RV'
 'Antonov An-2T' 'Antonov An-32' 'Tupolev TU-134A / Antonov An-26'
 'Antonov An-12AP' 'Antonov An-26' 'Antonov 12BP' 'Antonov AN-32'
 'Antonov AN-8' 'Antonov AN-72' 'Antonov AN-30' 'Antonov 12BK' 'Antonov 32'
 'Antonov AN-28' 'Antonov AN-22A' 'Antonov AN-26B' 'Antonov AN-124'
 'Antonov AN-32B' 'Antonov AN-2' 'Antonov 26B' 'Antonov 2R' 'Antonov An-2R'
 'Antonov 12B' 'Antonov An-124' 'Antonov AN-12B' 'Antonov An-32B'
 'Antonov AN-124-100' 'Antonov An-72' 'Antonov 12' 'Antonov 

Vemos que solo para 'Antonov' aparecen 76 tipos diferentes, debido a las inconsistencias en nomenclatura. Por ejemplo, 'Antonov AN-10' aparece como 'Antonov AN-10', 'Antonov AN-10A' 'Antonov An-10A', 'Antonov An-10', es decir, cuatro referencias distintas para el mismo modelo (o variantes del modelo).

Intentar dar consistencia a todos los valores de esta columna sería lo ideal, pero es un proceso muy laborioso. Es por ello que vamos a centrarnos en los tipos de avión que se vieron implicados en un mayor número de accidentes.

In [639]:
mas_accidentados = (data.groupby('Type')['Fatalities'].count()).sort_values(ascending = False)
aviones_mas_accidentados = list(mas_accidentados.index.values[0:30])
aviones_mas_accidentados

['Douglas DC-3',
 'de Havilland Canada DHC-6 Twin Otter 300',
 'Douglas C-47A',
 'Douglas C-47',
 'Douglas DC-4',
 'Yakovlev YAK-40',
 'Antonov AN-26',
 'Junkers JU-52/3m',
 'Douglas C-47B',
 'Douglas DC-6B',
 'De Havilland DH-4',
 'Breguet 14',
 'Curtiss C-46A',
 'Douglas C-47-DL',
 'Douglas DC-6',
 'McDonnell Douglas DC-9-32',
 'Antonov AN-24',
 'Antonov AN-12',
 'Curtiss C-46',
 'Embraer 110P1 Bandeirante',
 'Junkers F-13',
 'Tupolev TU-134A',
 'Fokker F-27 Friendship 600',
 'de Havilland Canada DHC-6 Twin Otter 100',
 'Cessna 208B Grand Caravan',
 'Lockheed C-130H',
 'Fokker F-27 Friendship 200',
 'de Havilland Canada DHC-6 Twin Otter 200',
 'Britten-Norman BN-2A Islander',
 'Lockheed 18 Lodestar']

De ellos, 'de Havilland Canada DHC-6 Twin Otter 300', 'de Havilland Canada DHC-6 Twin Otter 100' y 'de Havilland Canada DHC-6 Twin Otter 200', son variantes del mismo modelo. Lo mismo sucede con 'Douglas C-47A', 'Douglas C-47' y 'Douglas C-47-DL', los 'Fokker F-27 Friendship' y los 'Curtis C-46'.

In [669]:
def consistente(x):
    if (x == 'de Havilland Canada DHC-6 Twin Otter 100' or x == 'de Havilland Canada DHC-6 Twin Otter 200' or\
        x == 'de Havilland Canada DHC-6 Twin Otter 300'):
        x = 'de Havilland Canada DHC-6 Twin Otter'
    if (x == 'Douglas C-47A' or x == 'Douglas C-47' or x == 'Douglas C-47-DL'):
        x = 'Douglas C-47'
    if (x == 'Fokker F-27 Friendship 200' or x == 'Fokker F-27 Friendship 600'):
        x = 'Fokker F-27 Fiendship'
    if (x == 'Curtiss C-46' or x == 'Curtiss C-46A'):
        x = 'Curtiss C-46'
    return x

In [670]:
data['Type'] = data['Type'].map(lambda x: consistente(x))

Volvemos a calcular y reducimos a 9 tipos de avion.

In [674]:
mas_accidentados = (data.groupby('Type')['Fatalities'].count()).sort_values(ascending = False)
aviones_mas_accidentados = list(mas_accidentados.index.values[0:8])
aviones_mas_accidentados

['Douglas DC-3',
 'Douglas C-47',
 'de Havilland Canada DHC-6 Twin Otter',
 'Douglas DC-4',
 'Curtiss C-46',
 'Yakovlev YAK-40',
 'Antonov AN-26',
 'Fokker F-27 Fiendship']

Lo siguiente es obtener mediante scrapping el valor 'Role' para cada uno de estos tipos.

In [675]:
url_list = ['https://en.wikipedia.org/wiki/Douglas_DC-3',
            'https://en.wikipedia.org/wiki/Douglas_C-47_Skytrain',
            'https://en.wikipedia.org/wiki/De_Havilland_Canada_DHC-6_Twin_Otter',
            'https://en.wikipedia.org/wiki/Douglas_DC-4',
            'https://en.wikipedia.org/wiki/Curtiss_C-46_Commando',
            'https://en.wikipedia.org/wiki/Yakovlev_Yak-40',
            'https://en.wikipedia.org/wiki/Antonov_An-26',
            'https://en.wikipedia.org/wiki/Fokker_F27_Friendship']
role_list = []
for url in url_list:
    soup = BeautifulSoup(urlopen(url).read(), "lxml")
    table = soup.find('table')
    headers = table.find_all('th')
    for h in headers:
        if 'Role' in h:
            row = h.find_parent()
    role_list.append(str(row.getText()).split('\n')[2])
role_list

['Airliner and transport aircraft',
 'Military transport aircraft',
 'Utility aircraft',
 'Airliner/transport aircraft',
 'Military transport aircraft',
 'Regional jet/ VIP transport',
 'Transport aircraft',
 'Transport aircraft']

Creamos el dataframe que asocia a cada tipo con su role.

In [676]:
r = {'Type': aviones_mas_accidentados,
    'Role':role_list}
role_df = pd.DataFrame(r, columns = ['Type', 'Role'])
role_df

,Type,Role
0,Douglas DC-3,Airliner and transport aircraft
1,Douglas C-47,Military transport aircraft
2,de Havilland Canada DHC-6 Twin Otter,Utility aircraft
3,Douglas DC-4,Airliner/transport aircraft
4,Curtiss C-46,Military transport aircraft
5,Yakovlev YAK-40,Regional jet/ VIP transport
6,Antonov AN-26,Transport aircraft
7,Fokker F-27 Fiendship,Transport aircraft


Añadimos la nueva columna al dataframe existente de los accidentes de avión.

In [679]:
new_data = data.merge(role_df, how = 'outer')

In [680]:
print new_data.head()

         Date   Time                            Location  \
0  09/17/1908  17:18                 Fort Myer, Virginia   
1  07/12/1912  06:30             AtlantiCity, New Jersey   
2  08/06/1913    NaN  Victoria, British Columbia, Canada   
3  09/09/1913  18:30                  Over the North Sea   
4  10/17/1913  10:30          Near Johannisthal, Germany   

                 Operator Flight #          Route                    Type  \
0    Military - U.S. Army      NaN  Demonstration        Wright Flyer III   
1    Military - U.S. Navy      NaN    Test flight               Dirigible   
2                 Private        -            NaN        Curtiss seaplane   
3  Military - German Navy      NaN            NaN  Zeppelin L-1 (airship)   
4  Military - German Navy      NaN            NaN  Zeppelin L-2 (airship)   

  Registration cn/In  Aboard  Fatalities  Ground  \
0          NaN     1     2.0         1.0     0.0   
1          NaN   NaN     5.0         5.0     0.0   
2          NaN   NaN

Vemos que se ha añadido la columna 'Role'. Comprobamos que contiene los valores obtenidos para los tipos de avión elegidos.

In [682]:
new_data[new_data['Type'].str.contains('de Havilland Canada DHC-6 Twin Otter') == True]

,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary,Role
2652,03/11/1967,NaN,"Mt Visenti, Italy",Aeralpi,NaN,Venezia - Belluno,de Havilland Canada DHC-6 Twin Otter,I-CLAI,30,5.0,4.0,0.0,Flew into Colcanin Mountain in heavy fog while...,Utility aircraft
2653,11/23/1968,19:59,"Santa Ana, California",Cable Commuter,544,Los Angeles - Santa Ana,de Havilland Canada DHC-6 Twin Otter,N7666,148,9.0,9.0,0.0,"While on approach to the airport, in dense fog...",Utility aircraft
2654,07/15/1969,06:57,"New York City, New York",New York Airways,NaN,New York - Newark,de Havilland Canada DHC-6 Twin Otter,N558MA,175,14.0,3.0,0.0,The plane flew into wake turbulence and crashe...,Utility aircraft
2655,02/10/1970,18:37,"Long Island Sound, Connecticut",Pilgrim Airlines,203,"Groton, CT - New York, NY",de Havilland Canada DHC-6 Twin Otter,N124PM,41,5.0,5.0,0.0,Crashed into Long Island Sound while on a flig...,Utility aircraft
2656,04/28/1970,NaN,"Kainatu, New Guinea",Trans Australia Airlines,NaN,NaN,de Havilland Canada DHC-6 Twin Otter,VH-TGR,6,11.0,8.0,0.0,Crashed after taking off from a dirt air strip.,Utility aircraft
2657,05/27/1972,NaN,"Near Helgoland, Germany",General Air,NaN,NaN,de Havilland Canada DHC-6 Twin Otter,D-IDHC,31,13.0,8.0,0.0,Crashed after taking off this North sea island...,Utility aircraft
2658,07/11/1972,16:00,"Near Grytoya, Norway",Military - Royal Norwegian Air Force,NaN,Bardufoss - Bodoe,de Havilland Canada DHC-6 Twin Otter,67-056,56,19.0,19.0,0.0,Crashed in rain and fog on the island of Hinno...,Utility aircraft
2659,12/21/1972,NaN,"Off St. Maarten off, Netherlands Antilles",Air Guadeloupe,NaN,Guadeloupe - St. Maarten,de Havilland Canada DHC-6 Twin Otter,F-OGFE,258,13.0,13.0,0.0,Crashed on approach into the sea.,Utility aircraft
2660,02/28/1973,NaN,"Near Nabire, Indonesia",Merpati Nusantara Airlines,NaN,NaN,de Havilland Canada DHC-6 Twin Otter,PK-NUC,70,13.0,13.0,0.0,Crashed while en route.,Utility aircraft
2661,04/18/1973,NaN,"Bamiyan, Afghanistan",Bakhtar Afghan Airlines,NaN,Bamiyan - Kabul,de Havilland Canada DHC-6 Twin Otter,YA-GAT,111,19.0,4.0,0.0,The chartered American/Canadian tour group's p...,Utility aircraft
